In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")]
messages.append(HumanMessage(content=f"Yes, that's right.",name="Lance"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for m in messages:
    m.pretty_print()

In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm= ChatOpenAI(model="gpt-4o")
result= llm.invoke(messages)
type(result)

## Tools

In [ ]:
def multiply(a: int, b: int)->int:
    """Multiply a and b
    
    Args:
        a: first int
        b: second int
    """

    return a * b

llm_with_tools= llm.bind_tools([multiply])

In [ ]:
tool_call= llm_with_tools.invoke([HumanMessage(content=f"What is 2 multiplied by 3", name= "Lance")])
tool_call

## Usando mensagens como estado

In [ ]:
from typing import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

## Reducers

In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

Como ter uma lista de mensagens no estado do grafo é muito comum, o LangGraph já vem com um MessagesState pré-construído!

O MessagesState é definido:

Com uma chave única de mensagens pré-construída

Esta é uma lista de objetos AnyMessage

Ele usa o reducer add_messages

Normalmente usamos o MessagesState porque ele é menos verboso do que definir um TypedDict personalizado, como mostrado acima.

In [ ]:
from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    # Add any keys needed beyond messages, which is pre-built 
    pass

In [ ]:
# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on marine biology.", name="Lance")
                   ]

# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Test
add_messages(initial_messages , new_message)

## Graph

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
    
# Node
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)
graph = builder.compile()


In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 2 and 3")})
for m in messages['messages']:
    m.pretty_print()

## Resumo da Diferença (Entrada/Criação)

<table>
    <thead>
        <tr>
            <th>Método de Definição</th>
            <th>O que você cria para o invoke()</th>
            <th>Formato da Entrada para graph.invoke()</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>@dataclass</td>
            <td>Uma instância da classe</td>
            <td>Objeto de classe: <code>graph.invoke(DataclassState(...))</code></td>
        </tr>
        <tr>
            <td>Pydantic (BaseModel)</td>
            <td>Uma instância da classe</td>
            <td>Objeto de classe: <code>graph.invoke(PydanticState(...))</code></td>
        </tr>
        <tr>
            <td>TypedDict</td>
            <td>Um dicionário</td>
            <td>Dicionário: <code>graph.invoke({"key": value, ...})</code></td>
        </tr>
    </tbody>
</table>

Tanto dataclass quanto Pydantic incentivam o paradigma da Programação Orientada a Objetos (criação de instâncias).

## 🔄 Similaridade (Saída/Atualização)

Apesar da diferença na forma como o estado é criado inicialmente, na saída de um nó (o que você return da função do nó), você sempre retorna um dicionário que representa as mudanças incrementais de estado.

Isso acontece porque o LangGraph usa o operador de união para mesclar o estado retornado com o estado anterior:

* Estado Anterior: LangGraph mantém o estado completo (seja ele internamente um objeto ou um dicionário).

* Retorno do Nó: Seu nó retorna um dicionário (ex: {"mood": "happy"}).

* Fusão: O LangGraph funde (merge) esse dicionário com o estado anterior. Se você usou uma classe (Pydantic/Dataclass), o LangGraph garante que a fusão respeite o esquema da classe.

Isso significa que, independentemente do tipo de estado que você escolher:

* Para Ler: Você usa o formato de acesso do tipo (state.key ou state["key"]).

* Para Atualizar: Você sempre retorna um dicionário contendo apenas as chaves que deseja alterar.